In [4]:
import numpy as np
import pandas as pd
from selenium import webdriver

In [5]:
browser = webdriver.Chrome(r"C:\Users\eakhumb\Downloads\chromedriver_win32\chromedriver.exe")
browser.get("https://stats.nba.com/players/traditional")
browser.maximize_window()
pages = browser.find_element_by_xpath("//select[@class='stats-table-pagination__select ng-pristine ng-untouched ng-valid ng-not-empty']").text[-1]
data = pd.DataFrame()
pages = int(pages)-1
while(pages):
    div_table = browser.find_element_by_class_name("nba-stat-table__overflow")
    table = div_table.find_element_by_tag_name("table")
    temp = data
    data = pd.read_html(table.get_attribute('outerHTML'))[0]
    data = data.append(temp)
    pages = pages-1
    browser.find_element_by_xpath("//div[@class='stats-table-pagination__inner stats-table-pagination__inner--bottom']").find_element_by_class_name("stats-table-pagination__next").click()
browser.close()

In [6]:
fulldata = data

In [8]:
data = data[data["GP"]>=data["GP"].max()//2]
data = data.iloc[:,1:list(data.columns).index("+/-")+1]

In [9]:
MIN_multi = 100/data["MIN"].mean()
PTS_multi = 100/data["PTS"].mean()
OREB_multi = 100/data["OREB"].mean()
DREB_multi = 100/data["DREB"].mean()
REB_multi = 100/data["REB"].mean()
AST_multi = 100/data["AST"].mean()
TOV_multi = 100/data["TOV"].mean()
STL_multi = 100/data["STL"].mean()
BLK_multi = 100/data["BLK"].mean()
DD2_multi = 100/data["DD2"].mean()
TD3_multi = 100/data["TD3"].mean()
box_multi = 100/data["+/-"].mean()

In [10]:
def score_all(data):
    return ((data["PTS"]*PTS_multi)+(data["REB"]*REB_multi)+(data["AST"]*AST_multi)-(data["TOV"]*TOV_multi)+(data["STL"]*STL_multi)+(data["BLK"]*BLK_multi))
def best_def(data):
    return ((data["STL"]*STL_multi)+(data["BLK"]*BLK_multi))
def best_off(data):
    return ((data["PTS"]*PTS_multi)+(data["REB"]*REB_multi)+(data["AST"]*AST_multi)-(data["TOV"]*TOV_multi))

In [11]:
newdata = data

In [12]:
newdata["score"] = newdata.apply(lambda x:score_all(x), axis=1)
newdata = newdata.sort_values("score", ascending=False)
newdata["score_rank"] = (newdata.reset_index().drop(columns="index").index)+1
newdata["def_score"] = newdata.apply(lambda x:best_def(x), axis=1)
newdata = newdata.sort_values("def_score", ascending=False)
newdata["def_rank"] = (newdata.reset_index().drop(columns="index").index)+1
newdata["off_score"] = newdata.apply(lambda x:best_off(x), axis=1)
newdata = newdata.sort_values("off_score", ascending=False)
newdata["off_rank"] = (newdata.reset_index().drop(columns="index").index)+1

In [13]:
def over_rank(data):
    return ((data["score_rank"]+data["def_rank"]+data["off_rank"])/3)
def offdef_rank(data):
    return ((data["def_rank"]+data["off_rank"])/2)

In [15]:
newdata["over_rank"] = newdata.apply(lambda x:over_rank(x), axis=1)
newdata["offdef_rank"] = newdata.apply(lambda x:offdef_rank(x), axis=1)
newdata = newdata.sort_values("score", ascending=False)

In [16]:
newdata.to_excel(r"C:\Users\eakhumb\Downloads\NBA.xlsx", index=False)